In [ ]:
# Install dependencies
%pip install anthropic python-dotenv

In [ ]:
# load env variables
from dotenv import load_dotenv
load_dotenv()

In [ ]:
# create and API client
from anthropic import Anthropic
client = Anthropic() 

model = "claude-4-5-sonnet" 

In [ ]:
# Helper functions to manage messages
def add_user_message(messages, content):
    user_message= { "role": "user", "content": content}
    messages.append(user_message)

def add_assistant_message(messages, content):
    assistant_message= { "role": "assistant", "content": content}
    messages.append(assistant_message)

system_prompt = """
You are a turist guide assistant. 
Help users by providing information about popular travel destinations, attractions, and travel tips.
"""

def create_chat_completion(client, model, messages, system_prompt=None, temperature=0.7):
    params = {
        "model": model,
        "max_tokens": 1000,
        "messages": messages,
        "temperature": temperature,
    }

    if system_prompt:
        params["system"] = system_prompt

    response = client.messages.create(**params) 
    return response.content[0].text



In [ ]:
# Start with an empty message list
messages = []

# Add the initial user question
add_user_message(messages, "What is the capital of France?")

# Get Claude's response
answer = create_chat_completion(client, model, messages, system_prompt=system_prompt)

# Add Claude's response to the conversation history
add_assistant_message(messages, answer)
print("Claude's answer:", answer)

# Add a follow-up question
add_user_message(messages, "what are places to visit there?")

# Get Claude's response to the follow-up
answer = create_chat_completion(client, model, messages, system_prompt=system_prompt)

# Add Claude's response to the conversation history
add_assistant_message(messages, answer)
print("Claude's follow-up answer:", answer)

In [ ]:
# Interactive chat loop with user input
messages = []

while True:
    user_input = input("You: ")
    print("You: ", user_input)

    #add user input to list of messages
    add_user_message(messages, user_input);

    #get response from claude
    answer = create_chat_completion(client, model, messages)

    #add claude response to list of messages
    add_assistant_message(messages, answer);
    print("----")
    print("Claude: ", answer)
    print("----")
    

In [ ]:
# To enable streaming, add stream=True to your messages.create call:
messages = []
add_user_message(messages, "Write a 1 sentence description of a fake database")

with client.messages.stream(
    model=model,
    max_tokens=1000,
    messages=messages
) as stream:
    for text in stream.text_stream:
        # Send each chunk to your client
        pass
    
    # Get the complete message for database storage
    final_message = stream.get_final_message()